In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import cmocean
from matplotlib import pyplot as plt

In [3]:
import os
import pathlib
import sys
from subprocess import call

In [4]:
from datetime import datetime

In [5]:
from datetime import datetime, timedelta

In [6]:
import numpy as np
import pandas as pd
import xarray as xr

In [7]:
sys.path.append("../code/")

In [8]:
import src

### parameters for papermill 

In [28]:
regions = ["NZ", "Ninos", "IOD"]
region = "NZ"
dpath = "/media/nicolasf/END19101/data/OISST/daily"
clim_path = "/home/nicolasf/operational/OISST_indices/outputs/"
shapes_path = "/home/nicolasf/operational/OISST_indices/data/shapefiles/"
ndays_agg = 1
ndays_back = 180
quantile = 0.9
lag = 0

### get the pathlib paths for the data and the climatology 

In [29]:
dpath = pathlib.Path(dpath).joinpath(region)
clim_path = pathlib.Path(clim_path).joinpath(region)
shapes_path = pathlib.Path(shapes_path)

### get the current date 

In [11]:
current_date = datetime.utcnow()

### get the first day of the period to extract 

In [12]:
first_day = current_date - timedelta(days=ndays_back)

## get the years to read 

In [13]:
years_to_get = np.unique(np.arange(first_day.year, current_date.year + 1))

In [14]:
lfiles = [dpath.joinpath(f"sst.day.mean.{year}.v2.nc") for year in years_to_get]

## opens the near realtime dataset 

In [15]:
dset = xr.open_mfdataset(lfiles, parallel=True, combine="by_coords")

In [16]:
dset

<xarray.Dataset>
Dimensions:  (time: 220, lat: 80, lon: 72)
Coordinates:
  * time     (time) datetime64[ns] 2022-01-01 2022-01-02 ... 2022-08-08
  * lat      (lat) float32 -49.88 -49.62 -49.38 -49.12 ... -30.62 -30.38 -30.12
  * lon      (lon) float32 162.1 162.4 162.6 162.9 ... 179.1 179.4 179.6 179.9
Data variables:
    sst      (time, lat, lon) float32 dask.array<chunksize=(220, 80, 72), meta=np.ndarray>

### get rid of the 29th of Feb if present 

In [17]:
dset = dset.convert_calendar("noleap")

### opens the climatology 

In [18]:
clim = xr.open_zarr(
    clim_path.joinpath(f"{region}_OISST_{ndays_agg}days_climatology_15_window.zarr")
)

In [19]:
clim

<xarray.Dataset>
Dimensions:    (lat: 80, lon: 72, dayofyear: 365, quantile: 5)
Coordinates:
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
  * lat        (lat) float32 -49.88 -49.62 -49.38 ... -30.62 -30.38 -30.12
  * lon        (lon) float32 162.1 162.4 162.6 162.9 ... 179.1 179.4 179.6 179.9
  * quantile   (quantile) float64 0.1 0.25 0.5 0.75 0.9
Data variables:
    average    (lat, lon, dayofyear) float32 dask.array<chunksize=(20, 36, 183), meta=np.ndarray>
    quantiles  (dayofyear, quantile, lat, lon) float64 dask.array<chunksize=(92, 2, 20, 36), meta=np.ndarray>
    std        (lat, lon, dayofyear) float32 dask.array<chunksize=(20, 36, 183), meta=np.ndarray>

### calculate the anomalies with respect to the 1991-2020 average

In [20]:
anoms = dset.groupby(dset.time.dt.dayofyear) - clim["average"]

### repeat the climatology over the time dimension 

In [21]:
clim_repeat = clim.sel(dayofyear=dset.time.dt.dayofyear)

In [22]:
clim_repeat

<xarray.Dataset>
Dimensions:    (lat: 80, lon: 72, time: 220, quantile: 5)
Coordinates:
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 213 214 215 216 217 218 219 220
  * lat        (lat) float32 -49.88 -49.62 -49.38 ... -30.62 -30.38 -30.12
  * lon        (lon) float32 162.1 162.4 162.6 162.9 ... 179.1 179.4 179.6 179.9
  * quantile   (quantile) float64 0.1 0.25 0.5 0.75 0.9
  * time       (time) object 2022-01-01 00:00:00 ... 2022-08-08 00:00:00
Data variables:
    average    (lat, lon, time) float32 dask.array<chunksize=(20, 36, 183), meta=np.ndarray>
    quantiles  (time, quantile, lat, lon) float64 dask.array<chunksize=(92, 2, 20, 36), meta=np.ndarray>
    std        (lat, lon, time) float32 dask.array<chunksize=(20, 36, 183), meta=np.ndarray>

### derive the mask 

In [23]:
mask = dset["sst"].where(dset["sst"] >= clim_repeat["quantiles"].sel(quantile=quantile))
mask = mask.where(np.isnan(mask), other=1)

In [ ]:
reads the shapefiles and 

In [26]:
import regionmask

In [27]:
import geopandas as gpd

In [37]:
NNI = gpd.read_file(shapes_path.joinpath('NNI_buffered_50km.shp')).to_crs('EPSG:4326')
WNI = gpd.read_file(shapes_path.joinpath('WNI_buffered_50km.shp')).to_crs('EPSG:4326')
ENI = gpd.read_file(shapes_path.joinpath('ENI_buffered_50km.shp')).to_crs('EPSG:4326')
NSI = gpd.read_file(shapes_path.joinpath('NSI_buffered_50km.shp')).to_crs('EPSG:4326')
WSI = gpd.read_file(shapes_path.joinpath('WSI_buffered_50km.shp')).to_crs('EPSG:4326')
ESI = gpd.read_file(shapes_path.joinpath('ESI_buffered_50km.shp')).to_crs('EPSG:4326')

In [38]:
dset

<xarray.Dataset>
Dimensions:  (time: 220, lat: 80, lon: 72)
Coordinates:
  * time     (time) object 2022-01-01 00:00:00 ... 2022-08-08 00:00:00
  * lat      (lat) float32 -49.88 -49.62 -49.38 -49.12 ... -30.62 -30.38 -30.12
  * lon      (lon) float32 162.1 162.4 162.6 162.9 ... 179.1 179.4 179.6 179.9
Data variables:
    sst      (time, lat, lon) float32 dask.array<chunksize=(220, 80, 72), meta=np.ndarray>

In [39]:
anoms

<xarray.Dataset>
Dimensions:    (time: 220, lat: 80, lon: 72)
Coordinates:
  * time       (time) object 2022-01-01 00:00:00 ... 2022-08-08 00:00:00
  * lat        (lat) float32 -49.88 -49.62 -49.38 ... -30.62 -30.38 -30.12
  * lon        (lon) float32 162.1 162.4 162.6 162.9 ... 179.1 179.4 179.6 179.9
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 213 214 215 216 217 218 219 220
Data variables:
    sst        (time, lat, lon) float32 dask.array<chunksize=(183, 20, 36), meta=np.ndarray>

In [40]:
mask

<xarray.DataArray 'sst' (time: 220, lat: 80, lon: 72)>
dask.array<where, shape=(220, 80, 72), dtype=float32, chunksize=(92, 20, 36), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) object 2022-01-01 00:00:00 ... 2022-08-08 00:00:00
  * lat        (lat) float32 -49.88 -49.62 -49.38 ... -30.62 -30.38 -30.12
  * lon        (lon) float32 162.1 162.4 162.6 162.9 ... 179.1 179.4 179.6 179.9
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 213 214 215 216 217 218 219 220
    quantile   float64 0.9
Attributes:
    long_name:     Daily Sea Surface Temperature
    units:         degC
    valid_range:   [-3. 45.]
    precision:     2.0
    dataset:       NOAA High-resolution Blended Analysis
    var_desc:      Sea Surface Temperature
    level_desc:    Surface
    statistic:     Mean
    parent_stat:   Individual Observations
    actual_range:  [-1.8  35.71]
    _ChunkSizes:   [   1  720 1440]